# Все импорты

In [7]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
from html import unescape
from bs4 import BeautifulSoup
import re
from tqdm.auto import tqdm
import unicodedata
import sqlite3
import time
import random
import pandas as pd

# Парсинг четырех новых медиа социнальной тематики.

## _Батенька, да Вы трансформер!_

Функция batenka_parse(page_num) выкачивает ссылки на статьи из архива издания "Батенька, да вы трансформер". На вход принимает номер страницы.

In [18]:
def batenka_parse(page_num):    
    url = f'https://batenka.ru/archive/?page={page_num}'
    req = requests.get(url, headers={'User-Agent': ua.random})
    html = req.text
    soup_1 = BeautifulSoup(html, 'html.parser')
    blocks = soup_1.find_all('a', {'class': 'anons__title-link'})
    hrefs = []
    for b in blocks:
        hrefs.append(b['href'])
    return hrefs

Функция batenka_article_parse(href) парсит веб-страницу со статьёй (историей) и выводит очищенный от тегов и прочего "мусора" текст. На вход принимает правую часть ссылки.

In [19]:
def batenka_article_parse(href):
    article_url = 'https://batenka.ru' + href
    req = requests.get(article_url, headers={'User-Agent': ua.random})
    article = req.text
    soup = BeautifulSoup(article, 'html.parser')
    text_block = soup.find_all('div', {'class': 'content-grid__order-column content-grid__column-half'})
    text = []
    for b in text_block:
        soup_1 = BeautifulSoup(str(b), 'html.parser')
        t = soup_1.get_text()
        t = unicodedata.normalize("NFKD", t)
        t = t.replace('\n','')
        t = t.replace('\t','')
        t = re.sub(r'([\.\?\!\:])([—\w])', r'\1 \2', t)
        t = re.sub(r'([\.\?\!\:])([—\w])', r'\1 \2', t)
        text.append(t)
    text = ' '.join(text)
    return text


Функция batenka_main() объединяет в себе функции batenka_parse(page_num) и batenka_article_parse(href), на выходе предоставляет DataFrame (df_batenka), с уникальным id текста, самим текстом и ссылкой на его источник.

In [20]:
def batenka_main():    
    hrefs = []
    for n in range(1, 3):
        hrefs.extend(batenka_parse(n))
    texts_lst = []
    counter = 0
    for h in tqdm(hrefs):
        text = batenka_article_parse(h)
        if text != '':
            dict_text = {}
            dict_text['id_text'] = 'Б' + str(counter)
            dict_text['text'] = text
            dict_text['link'] = 'https://batenka.ru' + h
            texts_lst.append(dict_text)
            counter += 1
    df_batenka = pd.DataFrame(texts_lst)
    return df_batenka

In [21]:
df_batenka = batenka_main()

In [61]:
df_batenka

,id_text,text,link
0,Б0,Так началась моя рабочая жизнь. Теперь мой бу...,https://batenka.ru/protection/police/psychiatr...
1,Б1,Прошла неделя. Я развлекал Келли в течение дня...,https://batenka.ru/resource/foreign-grooms-in-...
2,Б2,"Меня зовут Анна, мне тридцать три года, и я оч...",https://batenka.ru/protection/forest-that-no-o...
3,Б3,Уже через пару месяцев в новой стране соседка...,https://batenka.ru/resource/sexy/european-stri...
4,Б4,"— Начальник, ну давай его ёбнем, а? По-тихом...",https://batenka.ru/resource/med/quitting-psych...
...,...,...,...
71,Б71,Середина февраля. Meduza Bar в Берлине. Пять с...,https://batenka.ru/worship/standup-is-a-new-rap/
72,Б72,"За эти месяцы первый этаж нашего магазина, ст...",https://batenka.ru/unity/letter-from-reykjavik/
73,Б73,"Меня зовут Андри Магнасон, я писатель и экоакт...",https://batenka.ru/worship/glacier-funeral/
74,Б74,Русские хакеры так часто появляются в новостях...,https://batenka.ru/protection/gray-hackers/


## _Knife Media_  лонгриды

Эта функция парсит последовательно пять страниц в категории лонгридов Ножа, извлекая из них всю доступную информацию о статьях.

In [24]:
def parse_knife_pages():
    page_number = 1
    knife_pages = []
    while True:
        url = f'https://knife.media/category/longreads/page/{page_number}/'
        if page_number < 6:
            req = session.get(url, headers=headers)
            page = req.text
            soup = BeautifulSoup(page, 'html.parser')
            knife_pages.append(soup)
            page_number += 1
        else:
            break
    return knife_pages

Запустим ее.

In [25]:
knife_pages = parse_knife_pages()

Эта функция извлекает из информации о статьях ссылки на эти статьи, возвращая список ссылок.

In [26]:
def find_url_knife_articles(knife_pages):
    url_knife_articles = []
    for page in knife_pages:
        if len(url_knife_articles) < 51:
            for post in page.find_all('a', {'class': 'unit__content-link'}):
                link = re.compile('<a class="unit__content-link" href="(.*?)"')
                if link is not None:
                    p = str(post)
                    b = link.findall(p)
                    href = ''.join(b)
                    url_knife_articles.append(href)
        else:
            break
    return url_knife_articles

Запускаем.

In [27]:
url_knife_articles = find_url_knife_articles(knife_pages)

Эта функция парсит статьи, извлекая из них только текст, очищенный ото всех тегов и лишних знаков, и возвращает список статей.

In [28]:
def parse_knife_articles(url_knife_articles):
    khife_soup = []
    knife_articles = []
    for url in tqdm(url_knife_articles):
        text_post = session.get(url, headers=headers)
        page_post = text_post.text
        soup_post = BeautifulSoup(page_post, 'html.parser')
        #khife_soup.append(soup_post)
        soup = soup_post.find_all('script', {'type': 'application/ld+json'})
        s = str(soup)
        needed = re.compile('","text":"(.*?)"}</script>]')
        text = re.findall(needed, s)
        article = ''.join(text)
        a = article.replace('\xa0', ' ')
        knife_articles.append(a)

    return knife_articles

In [29]:
knife_articles = parse_knife_articles(url_knife_articles)

Здесь мы получаем уникальный айди для каждого извлеченного текста

In [30]:
list_id = []
for i in range(1, (len(knife_articles) + 1)):
    id_text = 'k' + str(i)
    list_id.append(id_text)

Записываем в датафрейм полученный айди, текст каждой статьи и ссылку на нее.

In [31]:
df_knife = pd.DataFrame()
df_knife['id_text'] = list_id
df_knife['text'] = knife_articles
df_knife['link'] = url_knife_articles

In [32]:
df_knife

,id_text,text,link
0,k1,"Когда Тордис Эльве было шестнадцать, она подве...",https://knife.media/violence-and-forgiveness/
1,k2,В современном обществе становится всё меньше п...,https://knife.media/eulogy/
2,k3,Александр Николаевич Даценко — геолог с двадца...,https://knife.media/gold-in-taiga/
3,k4,"Чтобы защититься от мертвой ведьмы, Хома Брут,...",https://knife.media/disaster-rituals/
4,k5,Не так давно была раскрыта крупнейшая криминал...,https://knife.media/paul-le-roux-empire/
5,k6,Люди могут заглушать неприятные эмоции по-разн...,https://knife.media/i-dont-feel-anything/
6,k7,Могут ли чат-боты познавать? Ответ на этот воп...,https://knife.media/against-chaos/
7,k8,"Достаточно ли один раз покурить траву, чтобы с...",https://knife.media/outsider-theory/
8,k9,Интроспекция — наблюдение человека за собствен...,https://knife.media/mironov/
9,k10,"Дружба в детстве — казалось бы, что может быть...",https://knife.media/what-is-your-name/


# _Холод Медиа_

Получаем доступ к сайту.

In [34]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
url = f'https://holod.media/#!/tab/141389865-2'
req = session.get(url, headers={'User-Agent': ua.random})

С нескольких основных страниц последовательно извлекаем список с ссылками на все доступные истории.

In [35]:
def get_list_of_links(req):
    list_of_links = []
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block = soup.find_all('div', {'id':"rec148693382"})
    soup2 = BeautifulSoup(str(block), 'html.parser')
    for link in soup2.find_all('a'):
        pot_link = link.get('href')
        if pot_link.startswith('/'):
            list_of_links.append(pot_link)
    return list_of_links

In [36]:
list_of_links = get_list_of_links(req)

Переходим по каждой из полученных ссылок, извлекаем из всего тела статьи исключительно ее текст.

In [37]:
def get_story(link_to_story):
    url = f'https://holod.media/{link_to_story}'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    all_text = soup.find_all('div', {'field': 'text'})
    clear_text = ''
    for t in all_text:
        clear_t = re.sub(r'(<.*?>)', '', str(t))
        clear_text += clear_t
    return clear_text

Многократно применив описанную выше функцию, получаем список текстов статей.

In [38]:
list_of_stories = []
for link in list_of_links:
    list_of_stories.append(get_story(link))

Создаем айди для дф.

In [40]:
id_list = []
for i in range(1, (len(list_of_links) + 1)):
    id_text = 'h' + str(i)
    id_list.append(id_text)

Получаем список полных ссылок на каждую статью.

In [45]:
def get_full_links(list_of_links):
    list_of_full_links = []
    for i in list_of_links:
        list_of_full_links.append('https://holod.media'+i)
    return list_of_full_links

In [42]:
list_of_full_links = get_full_links(list_of_links)

Заполняем нашими списками датафрейм.

In [43]:
df_holod = pd.DataFrame()
df_holod['id_text'] = id_list
df_holod['text'] = list_of_stories
df_holod['link'] = list_of_full_links

In [44]:
df_holod

,id_text,text,link
0,h1,В Хабаровске 18 октября митингующие за освобож...,https://holod.media/khabarovsk-report
1,h2,В Железнодорожном районе Хабаровска в двухэтаж...,https://holod.media/khabarovsk-freddy
2,h3,После жестких задержаний на протестной акции в...,https://holod.media/timirov
3,h4,На 94-й день протестов в Хабаровске 64-летняя ...,https://holod.media/khabarovsk-egorovna
4,h5,"О том, что в телеграме есть сообщество фроттер...",https://holod.media/frottage
5,h6,"Россия — одна из немногих стран, где разрешено...",https://holod.media/surrogates
6,h7,В Нижнем Новгороде покончила с собой создатель...,https://holod.media/irina-slavina
7,h8,Повалихинское поселение в Костромской области ...,https://holod.media/glava-povalikhino
8,h9,Одно из отделений реанимации московской клинич...,https://holod.media/covid-again
9,h10,В России около 26% женщин так или иначе ограни...,https://holod.media/eat-me


# _Такие Дела_ портреты

Данная функция выводит список ссылок на каждую статью, информация о. которой содерджится на странице портретов Таких Дел, начиная со значения _start_ и заканчивая _end_.

In [47]:
def get_arts(start, end):
    links1 = []
    for i in tqdm(range(start,end)):
        url = f'https://takiedela.ru/rubrics/portret/page/{i}/'
        session = requests.session()
        ua = UserAgent(verify_ssl=False)
        req = session.get(url, headers={'User-Agent': ua.random})
        time.sleep(random.uniform(1.1, 5.2))
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')
        txt = soup.find_all('a', {'class': "b-material__txt b-material__txt_article"})
        txt1 = soup.find_all('a', {'class': "b-material__txt b-material__txt_article "})
        help_info = soup.find_all('a', {'class': "b-help-info b-help-info_article"})
        for i in txt:
            links1.append(i.attrs['href'])
        for i in txt1:
            links1.append(i.attrs['href'])
        for i in help_info:
            links1.append(i.attrs['href'])
    return links1

Эта функция возвращает список списков, состоящих из айди, текста статьи и ссылки на нее. Список списков готов к записи в датафрейм. Посколько сайт таких дел иногда чудит с кодировкой, поэтому я ввожу проверку также на кодировку и пустую строку, чтобы в нашей базе не было ненужного мусора.

In [48]:
def prep_db(links1):
    prep_db = [] 
    counter = 0
    for l in tqdm(links1):
        one_string = []
        req = session.get(l, headers={'User-Agent': ua.random})
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')
        all_texts = []
        text = soup.find_all('div', {'class': "b-single__content-720"})
        one_text = []
        one_list = []
        for i in text:
            w = i.get_text()
            if 'Ñ' not in w:
                one_text.append(w)
        if one_text != []:
            counter += 1
            clear = ''.join(one_text)
            all_texts.append(clear)
            alls = [x.replace('\n', ' ').replace('\xa0', ' ') for x in all_texts if x != '']
            one_list.append(f'td{counter}')
            one_list.append(alls)
            one_list.append(l)
            prep_db.append(one_list)
    return prep_db

Применяем первую функцию для страниц с 3-й по 12-ю, затем извлекаем статьи по полученным из этой функции ссылкам.

In [49]:
a = get_arts(2,11)

In [50]:
f = prep_db(a)

Записываем информацию из полученного ранее списка списков в датафрейм и выводим текст в более приличном виде.

In [58]:
import pandas as pd
df_td = pd.DataFrame.from_records(f, columns=['id_text', 'text', 'link'])[1:].copy()

In [59]:
df_td['text'] = [''.join(x) for x in df_td['text']]

In [60]:
df_td

,id_text,text,link
1,td2,Мы созваниваемся с Дмитрием по скайпу. С...,https://takiedela.ru/2020/07/psikhoanalitik-go...
2,td3,Абсолютная чемпионка мира по метанию нож...,https://takiedela.ru/2020/06/na-nozhakh/
3,td4,«Чемпион России по триатлону работает курьер...,https://takiedela.ru/2020/05/ya-nichego-ne-boyus/
4,td5,"Cлучилось все, как водится, неожиданно: две н...",https://takiedela.ru/2020/04/deputat-na-kolesakh/
5,td6,Неделю назад Таня защитила диплом в медицинск...,https://takiedela.ru/2020/07/socialnyy-lift-dl...
...,...,...,...
71,td72,"«Либо ты, либо ребенок». Катя не могла ...",https://takiedela.ru/2018/12/mama-kotoraya-dys...
72,td73,Гульнара Поздеева работает в «четырке» —...,https://takiedela.ru/2018/12/gulnara-reshaet-l...
73,td74,В этом месте нет ничего примечательного....,https://takiedela.ru/2018/11/khoroshego-vam-ra...
74,td75,Осенью 2014 года в одном из воронежских ...,https://takiedela.ru/2018/10/da-ya-prospal-yep...


# Собираем все в один датафрейм.

Собираем все имеющиеся датафреймы в один через concatenate. Устанавливаем индексы по айди текста.

In [54]:
frames = [df_knife, df_holod, df_batenka, df_td]
knife_holod_batenka_td = pd.concat(frames)
knife_holod_batenka_td.set_index('id_text')

,text,link
id_text,,
k1,"Когда Тордис Эльве было шестнадцать, она подве...",https://knife.media/violence-and-forgiveness/
k2,В современном обществе становится всё меньше п...,https://knife.media/eulogy/
k3,Александр Николаевич Даценко — геолог с двадца...,https://knife.media/gold-in-taiga/
k4,"Чтобы защититься от мертвой ведьмы, Хома Брут,...",https://knife.media/disaster-rituals/
k5,Не так давно была раскрыта крупнейшая криминал...,https://knife.media/paul-le-roux-empire/
...,...,...
td72,"«Либо ты, либо ребенок». Катя не могла ...",https://takiedela.ru/2018/12/mama-kotoraya-dys...
td73,Гульнара Поздеева работает в «четырке» —...,https://takiedela.ru/2018/12/gulnara-reshaet-l...
td74,В этом месте нет ничего примечательного....,https://takiedela.ru/2018/11/khoroshego-vam-ra...


Загружаем результат в базу данных SQL

In [64]:
con = sqlite3.connect('project_nlp.db')
c = con.cursor()
knife_holod_batenka_td.to_sql(name='df_texts', con=con, if_exists='replace')
con.commit()
con.close()

База данных всех текстов с ссылками на них и их айди готова!
